# How fast is data access?

When it comes to data, the performance of the CPU/Memory, disk and database are key. This script measures how fast your system performs on these parameters.

The whole script should run under a minute on most reasonably fast systems.

In [1]:
from __future__ import print_function

import io
import time
import sqlalchemy
import numpy as np
import pandas as pd
from pathlib import Path

class Timer:
    def __init__(self, msg):
        self.msg = msg

    def __enter__(self):
        self.start = time.clock()

    def __exit__(self, *args):
        self.end = time.clock()
        print('{:0.3f}s {:s}'.format(self.end - self.start, self.msg))

### CPU / RAM

This is a pure numerical computation on values in memory that computes the [eigenvalues](http://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.linalg.eig.html) of a random dataset.

In [2]:
# Initialise the same data every time
np.random.seed(0)
data = np.random.random((1000, 1000))

# Time the computation
with Timer('computation'):
    np.linalg.eig(data)

1.991s computation


### Disk

Let's time sequential writes and reads on the disk.

The best way to do this is via [disktt](https://www.google.com/search?q=disktt) on Windows and [dd on Linux](https://www.thomas-krenn.com/en/wiki/Linux_I/O_Performance_Tests_using_dd).

Below is a crude approximation in Python. Note: this is heavily influenced by OS disk caching.

In [3]:
# Change this to any folder in the drive you want to test
folder = Path('D:/')

In [4]:
# Test the speed of the hard disk at this folder
# ... with this string data
data = bytes('0123456789') * 100000000

# Run the test
path = folder / 'tempfile'
with path.open(mode='wb', buffering=0) as handle:
    with Timer('sequential disk write'):
        handle.write(data)
        
with path.open(mode='rb', buffering=0) as handle:
    with Timer('sequential disk read'):
        handle.read()
        
path.unlink()

18.606s sequential disk write
0.632s sequential disk read


### Database

This script tests the speed of MySQL. It assumes that a MySQL instance running on localhost and a database called `test` accessible to user `root` with no password. You can [change the connection string](http://docs.sqlalchemy.org/en/latest/core/engines.html#mysql) based on your configuration.


In [5]:
# This is for a local MySQL database called test that you can connect to as root with no password
engine = sqlalchemy.create_engine('mysql://root@localhost/test')

# Test the connection
connection = engine.connect()

In [6]:
# Setup the data structures
data = pd.DataFrame(np.random.randint(0, 1000, (1000000, 3)))

metadata = sqlalchemy.MetaData(bind=engine)
metadata.reflect()

# Drop benchmark table
if 'benchmark' in metadata.tables:
    metadata.tables['benchmark'].drop()

# Create benchmark table again as MyISAM
table = sqlalchemy.Table(
    'benchmark', metadata,
    sqlalchemy.Column('0', sqlalchemy.Integer),
    sqlalchemy.Column('1', sqlalchemy.Integer),
    sqlalchemy.Column('2', sqlalchemy.Integer),
    extend_existing=True,
    mysql_engine='MyISAM',
)
metadata.create_all()

In [7]:
with Timer('database write'):
    data.to_sql('benchmark', con=engine, if_exists='append', index=False)
    
with Timer('database read'):
    data = pd.read_sql('benchmark', con=engine)

15.506s database write
4.576s database read
